In [7]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.insert(0, "..")
from stamp import *
from physics import *
%load_ext autoreload
%autoreload 2
from roipoly import RoiPoly
%matplotlib widget

In [8]:
table_outflow = Table.read("../outflow_hops.csv", comment='#')
table_outflow_davis = Table.read("../outflow_davisnohops.csv", comment='#')
table_hops = Table.read("../../catalogs/hops.fits")
table_davis = Table.read("../../catalogs/davis09_jets_without_hops.txt", format='ascii.tab')
outflow_def = table_outflow[table_outflow['outflow'] == '1']
outflow_may = table_outflow[table_outflow['outflow'] == '?']
outflow_davis = table_outflow_davis[(table_outflow_davis['outflow'] == '1') | (table_outflow_davis['outflow'] == '?')]
# print(outflow_davis)
# print(outflow_def)
# print(outflow_may)
# print("total number (maybe and definitely): ", np.size(outflow_davis)+np.size(outflow_def)+np.size(outflow_may))
# print("total definite: ", np.size(outflow_davis[outflow_davis['outflow'] == '1']) + np.size(outflow_def))
# print(outflow_davis.length)
# print(np.size(outflow_def))
# print(table_outflow)
# print(table_outflow)
c12 = SpectralCube.read("../../cubes/mask_imfit_12co_pix_2_Tmb.fits")
c13 = SpectralCube.read("../../cubes/mask_imfit_13co_pix_2_Tmb_regrid12co.fits")
c18 = SpectralCube.read("../../cubes/mask_imfit_c18o_pix_2_Tmb.fits")

In [13]:
width=height=3*u.arcmin
plt.clf()

for i, row in enumerate(table_outflow):
    if row["hops"] != 158:
        continue
# for i, row in enumerate(outflow_davis):
#     if i != iwork:
#         continue
#     davis = row["davis"]
    hops = row["hops"]
    blue_vel, red_vel = row["blue_vel"], row["red_vel"]
    tanabe = row["tanabe"]
    note = row["notes"]
#     davis_row = table_davis[table_davis["SMZ"] == davis][0]
    hops_row = table_hops[table_hops["HOPS"] == hops][0]
    coord = SkyCoord(hops_row["RAJ2000"], hops_row["DEJ2000"], unit=u.deg)
#     coord = SkyCoord(davis_row["_RAJ2000"], davis_row["_DEJ2000"], unit=u.deg)
    
    sub12 = extract_subcube(c12, region_class=RectangleSkyRegion,
                      region_kwargs=dict(center=coord, width=width, height=height))
    sub13 = extract_subcube(c12, region_class=RectangleSkyRegion,
                      region_kwargs=dict(center=coord, width=width, height=height))
 
    rms12 = rms(sub12, velocity_range=[[-2,0],[18,20]]*u.km/u.s, sigma_clipped=True)
    rms13 = rms(sub13, velocity_range=[[0,2], [16,18]]*u.km/u.s)
    #PARSE BLUE/RED_VEL FROM TABLE
#     print(blue_vel, red_vel)
    if blue_vel == '-':
        fit_12 = extract_subcube(sub12, CircleSkyRegion,
                                region_kwargs={'center':coord, 'radius':15*u.arcsec})
        
        blue_vel, red_vel = calc_linewings(fit_12, nsigma_vel=2.)
        blue_vel, red_vel = blue_vel.to(u.km/u.s), red_vel.to(u.km/u.s)
#         print(blue_vel, red_vel)
    else:
        try:
            blue_vel, red_vel = float(blue_vel)*u.km/u.s, float(red_vel)*u.km/u.s
        except ValueError:
            blue_vel, red_vel = blue_vel.strip("[]").split(','), red_vel.strip("[]").split(',')
            blue_vel = [float(a) for a in blue_vel]*u.km/u.s
            red_vel = [float(a) for a in red_vel]*u.km/u.s
    
    print(blue_vel, red_vel)
    #MAKE OUTFLOW MASKS
    try:
        
        blue_slab = sub12.spectral_slab(sub12.spectral_extrema[0], blue_vel)
        red_slab = sub12.spectral_slab(red_vel, sub12.spectral_extrema[1])
    
    except ValueError:
        blue_slab = sub12.spectral_slab(blue_vel[0], blue_vel[1])
        red_slab = sub12.spectral_slab(red_vel[0], red_vel[1])
    
    sigmom0_blue = sigma_mom0(blue_slab, channel_sigma=rms12)
    sigmom0_red = sigma_mom0(red_slab, channel_sigma=rms12)
    mask_bluelobe = blue_slab.moment0() > 10*sigmom0_blue
    mask_redlobe = red_slab.moment0() > 10*sigmom0_red
    
    
    do_fit = 0
#     blue_vel=7*u.km/u.s
#     red_vel=14*u.km/u.s
    nsigma_vel = 2.
#     blue_vel = 4.3*u.km/u.s
#     red_vel = 10.5*u.km/u.s
    fit_radius = 15*u.arcsec
#     width = height = 15j*u.arcmin
    start = 5
    stop = 50.
    step = 5
    fig = plt.figure(figsize=(10,5))
    plot_finder(c12, coord=coord,
            fit_cube=c12, fit_radius=fit_radius,
            nsigma_vel=nsigma_vel, blue_vel=blue_vel, red_vel=red_vel,
            fit_spectrum=do_fit, channel_sigma=rms12,
            region_width=width, region_height=height,
            blue_levels=np.arange(start, stop+step, step), red_levels=np.arange(start, stop+step, step),
            show_stamp=True, show_catalogs=True, show_spectrum=True, show_fit=do_fit,
            show_fitcircle=False, show_vrange=True, interactive=False, show_name=False,
            show_contour=1, show_redblue=0, show_outflows=False, redblue_mode='subtract',
            imshow_kwargs={"cmap":"RdBu_r", "interpolation":"none"},
            blue_contour_kwargs={'colors':'blue', 'linewidths':1, 'alpha':0.6, 'zorder':3},
            red_contour_kwargs={'colors':'red', 'linewidths':1, 'alpha':0.6, 'zorder':3},
            fig=fig, savefig=False, verbose=False, autoguess=True,
            spec_label=r'$^{12}$CO', draw_roi=False,
            catalogs=["/Users/jesse/repos/outflows/catalogs/hops.fits",
                      "/Users/jesse/repos/outflows/catalogs/davis09_h2jets.fits",
                      "/Users/jesse/repos/outflows/catalogs/spitzer_orion.fit"],
            catalog_kwargs=[dict(marker="s", s=10, color='black', lw=1, zorder=3),
                        dict(marker="+", s=60, color='black', lw=1, zorder=3),
                        dict(marker=".", s=5, color='black', lw=1, zorder=3)]
            )
    print(hops)
    my_roi = RoiPoly(color='r')
#     lt.clf()
#     my_roi = RoiPoly(color='r')
    ax = fig.axes[0]
#     plt.show()
#     ax.imshow(mask_padded+mask_big, origin='lower')
#     ax.imshow(np.load("../masks/maskroi_hops{}_{}arcmin_red.npy".format(hops, width.value)), origin='lower')
#     plt.show()

    
#     plt.imshow(mask_bluelobe)
#     plt.show()
#     plt.imshow(mask_redlobe)
#     plt.show()

/Users/jesse/miniconda3/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/Users/jesse/miniconda3/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/Users/jesse/miniconda3/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/Users/jesse/miniconda3/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/Users/jesse/miniconda3/lib/python3.7/site-packages/scipy/ndimage/measurements.py:272: Deprecati

5.0 km / s 9.5 km / s


/Users/jesse/miniconda3/lib/python3.7/site-packages/ipywidgets/widgets/widget.py:411: DeprecationWarning: Passing unrecoginized arguments to super(FigureCanvasNbAgg).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


FigureCanvasNbAgg()

/Users/jesse/miniconda3/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/Users/jesse/miniconda3/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/Users/jesse/miniconda3/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/Users/jesse/miniconda3/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/Users/jesse/miniconda3/lib/python3.7/site-packages/scipy/ndimage/measurements.py:272: Deprecati

158


/Users/jesse/miniconda3/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/Users/jesse/miniconda3/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/Users/jesse/miniconda3/lib/python3.7/site-packages/scipy/ndimage/measurements.py:272: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return _nd_image.find_objects(input, max_label)


In [14]:
np.save("../masks/maskroi_hops{}_{}arcmin_red".format(hops,width.value), my_roi.get_mask(mask_redlobe))

plt.figure()
plt.imshow(np.load("../masks/maskroi_hops{}_{}arcmin_red.npy".format(hops,width.value)), origin='lower')
plt.show()
# plt.imshow(np.load("maskroi_hops78_8.0arcmin_blue.npy")|np.load("maskroi_hops78_8.0arcmin_red.npy"))

/Users/jesse/miniconda3/lib/python3.7/site-packages/ipywidgets/widgets/widget.py:411: DeprecationWarning: Passing unrecoginized arguments to super(FigureCanvasNbAgg).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


FigureCanvasNbAgg()

In [25]:
import numpy as np
import matplotlib.pyplot as plt
mask_5arcmin = np.load("../masks/maskroi_hops158_3.0arcmin_blue.npy")
mask_5arcmin2 = np.load("../masks/maskroi_hops158_3.0arcmin_red.npy")
# mask_big = np.load("../masks/maskroi_hops84_17.0arcmin_red.npy")
# print(mask_small.shape, mask_big.shape)
# mask_padded = np.pad(mask_small, (mask_big.shape[0] - mask_small.shape[0])//2, mode='constant', constant_values=False)
mask_5arcmin.shape
plt.figure()
# plt.imshow(mask_5arcmin[10:80,10:80]+mask_5arcmin2[10:80,10:80])

plt.show()
mask_blue = mask_5arcmin[10:80, 10:80]
mask_red = mask_5arcmin2[10:80, 10:80]
print(mask_red.shape)
# np.save("../masks/maskroi_hops158_3.0arcmin_blue.npy", mask_blue)
# np.save("../masks/maskroi_hops158_3.0arcmin_red.npy", mask_red)
# plt.figure()
# plt.imshow(mask_padded, origin='lower')
# plt.show()
# plt.figure()
# plt.imshow(mask_small)
# plt.show()
# np.save("../masks/maskroi_hops84_17.0arcmin_blue", mask_padded)


/Users/jesse/miniconda3/lib/python3.7/site-packages/ipywidgets/widgets/widget.py:411: DeprecationWarning: Passing unrecoginized arguments to super(FigureCanvasNbAgg).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


FigureCanvasNbAgg()

(60, 60)
